In [248]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [249]:
import torch
import torch.nn as nn
import math
import numpy as np
import torch.nn.functional as F

In [250]:
def scaled_dot_product(q, k, v, mask=None):
    # q, k ,v = 30 x 8 x 200 x 64
    d_k = q.shape[-1] # 64
    scaled = torch.matmul(q, k.transpose(-1,-2))//math.sqrt(d_k) # 30 x 8 x 200 x 200
    if mask is not None:
        scaled += mask  # 30 x 8 x 200 x 200
    
    attention = F.softmax(scaled, dim=-1) # 30 x 8 x 200 x 200
    values = torch.matmul(attention, v) # 30 x 8 x 200 x 64
    return values,attention 

In [251]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model # 512
        self.num_heads = num_heads # 8
        self.head_dim = d_model // num_heads # 64
        self.qkv_layer = nn.Linear(d_model, 3 * d_model) # 512 -> 1536
        self.linear_layer = nn.Linear(d_model, d_model) # 512 -> 512
        
        
    def forward(self,x,mask=None):
        batch_size, seq_len, d_model = x.shape # 30 x 200 x 512
        qkv = self.qkv_layer(x) # 30 x 200 x 1536
        qkv = qkv.reshape(batch_size,seq_len,self.num_heads, 3*self.head_dim) # 30 x 200 x 8 x 192
        qkv = qkv.permute(0,2,1,3) # 30 x 8 x 200 x 192
        q, k, v = qkv.chunk(3,dim=-1) # [30 x 8 x 200 x 64] q,k,v
        values, attentions = scaled_dot_product(q, k, v, mask=None) # values = 30 x 8 x 200 x 64 , attention = 30 x 8 x 200 x 200 
        values = values.reshape(batch_size,seq_len,self.num_heads*self.head_dim) # 30 x 200 x 512
        out = self.linear_layer(values) # 30 x 200 x 512
        return out
        

In [252]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward,self).__init__()
        self.linear1 = nn.Linear(d_model,hidden) # 512 x hidden
        self.linear2 = nn.Linear(hidden,d_model) # hidden x 512
        self.relu = nn.ReLU() 
        self.dropout = nn.Dropout(p=drop_prob)
        
    def forward(self,x): # 30 x 200 x 512
        x = self.linear1(x) # 30 x 512 x hidden
        x = self.relu(x) # 30 x 512 x hidden
        x = self.dropout(x) # 30 x 200 x hidden
        x = self.linear2(x) # 30 x 200 x 512
        return x

In [253]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape # [512]
        self.eps = eps 
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) # 512
        self.beta = nn.Parameter(torch.zeros(parameters_shape)) # 512
        
    def forward(self, inputs): # inputs = 30 x 200 x 512
        dims = [-(i+1) for i in range(len(self.parameters_shape))] # [-1]
        mean = inputs.mean(dim=dims, keepdim = True) # 30 x 200 x 1
        var = ((inputs-mean)**2).mean(dim=dims, keepdim=True) # 30 x 200 x 1
        std = (var + self.eps).sqrt() # 30 x 200 x 1
        y = (inputs-mean) / std # 30 x 200 x 512
        out = self.gamma * y + self.beta # 30 x 200 x 512
        return out

In [254]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer,self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
    
    def forward(self, x):
        residual_x = x # 30 x 200 x 512
        x = self.attention(x, mask=None) # 30 x 200 x 512
        x = self.dropout1(x) # 30 x 200 x 512
        x = self.norm1(x + residual_x) # 30 x 200 x 512 
        
        residual_x = x # 30 x 200 x 512
        
        x = self.ffn(x) # 30 x 200 x 512
        x = self.dropout2(x) # 30 x 200 x 512
        x = self.norm2(x + residual_x) # 30 x 200 x 512
        
        return x # 30 x 200 x 512

In [255]:
class Encoder(nn.Module):
    def __init__(self,d_model,ffn_hidden,num_head,drop_prob,num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model,ffn_hidden,num_heads,
                                                  drop_prob) for _ in range(num_layers)])
        # input is passed through 5 or 10 number of EncoderLayers in
        # sequential manner.
    
    def forward(self,x):
        x = self.layers(x) 
        return x

In [256]:
class MultiHeadCrossAttention(nn.Module):
    def __init__(self,d_model,num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model, 2 * d_model)
        self.q_layer = nn.Linear(d_model, d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
        
    def forward(self, x, y, mask=None):
        batch_size, seq_len, d_model = x.shape # 30 x 200 x 512
        kv = self.kv_layer(x) # 30 x 200 x 1536
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, seq_len, self.num_heads, 2 * self.head_dim) # 30 x 200 x 8 x 192
        q = q.reshape(batch_size, seq_len, self.num_heads, self.head_dim)
        
        kv = kv.permute(0,2,1,3) # 30 x 8 x 200 x 192
        q = q.permute(0,2,1,3)
        
        k, v = kv.chunk(2,dim=-1) # [30 x 8 x 200 x 64] k,v

        values, attentions = scaled_dot_product(q, k, v, mask) # values = 30 x 8 x 200 x 64 , attention = 30 x 8 x 200 x 200 
        values = values.reshape(batch_size,seq_len,self.num_heads*self.head_dim) # 30 x 200 x 512
        out = self.linear_layer(values) # 30 x 200 x 512
        return out

In [257]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer,self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization([d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)
    
    def forward(self, x, y, decoder_mask):
        _y = y
        print("Masked Self Attention")
        y = self.self_attention(x,mask=decoder_mask)
        print("Dropout 1")
        y = self.dropout1(y)
        print("Add + Layer Nomralization 1")
        y = self.norm1(y + _y)
        
        _y = y
        print("Cross Attention")
        y = self.encoder_decoder_attention(x, y, mask=None)
        

In [258]:
class SequentialDecoder(nn.Sequential):
    def forward(self,*inputs):
        x, y, mask = inputs
        for module in self._modules.values():
            y = module(x, y, mask)
        return y

In [259]:
class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers=1):
        super().__init__()
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])
        
    def forward(self,x,y,mask):
        # x : 30 x 200 x 512
        # y : 30 x 200 x 512
        # mask : 200 x 200
        y = self.layers(x,y,mask)
        return y

In [260]:
d_model = 512
batch_size = 30
num_heads = 8
drop_prob = 0.1
max_seq_len = 200
ffn_hidden = 2048
num_layers = 5
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [261]:
x = torch.randn((30,200,512))
out = encoder(x)

In [262]:
out.shape

torch.Size([30, 200, 512])

In [263]:
x = out
y = torch.randn((batch_size,max_seq_len,d_model))
mask = torch.full([max_seq_len, max_seq_len] , float('-inf'))
mask = torch.triu(mask, diagonal =1)
decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
out = decoder(x, y, mask)

Masked Self Attention
Dropout 1
Add + Layer Nomralization 1
Cross Attention
Masked Self Attention
Dropout 1
Add + Layer Nomralization 1


TypeError: unsupported operand type(s) for +: 'Tensor' and 'NoneType'